 Input Data | Output Data |
| --- | --- |
| Region | Tokyo - city/ward, district/town/village |
| Price range (Korean standards) | Accurate price (price) |
| Room Type | Private room, Entire home/apt |
| Accommodates number of guests | Host-written accommodation description (neighborhood_overview) |
| Bathrooms (bathrooms_text) | Host identity verification (host_identity_verified) |
| Bedrooms (bedrooms) | Customer-written accommodation description (description) |
| Beds: number of beds | Minimum number of nights (minimum_nights) |
| Rating (review_scores_rating) range | Overall rating score (review_scores_rating), Number of reviews in the last 30 days (number_of_reviews_l30d), Date of the last review (last_review), Date calendar was last updated (calendar_updated) |

In [13]:
import pandas as pd
import numpy as np
import random

In [14]:
data_file = '/Users/genie/Documents/COLLABORATION/AirbnbWise/Tokyo_Airbnb/data/listings.csv.gz'

listing_gz = pd.read_csv(data_file, compression='gzip', header=0, sep=',', quotechar='"')
listing_gz.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

### User Class

In [15]:
class User:
    def __init__(self, idx, listing_id, host_id, host_name, review_scores_rating, bedrooms, beds, bathrooms, region = None, price = None, accommodates = None, minimum_nights = None, id = None, password = None):
        self.idx = idx
        self.listing_id = listing_id
        self.host_id = host_id
        self.host_name = host_name
        self.review_scores_rating = review_scores_rating
        self.bedrooms = bedrooms
        self.beds = beds
        self.bathrooms = bathrooms
        self.region = region
        self.price = price
        self.accommodates = accommodates
        self.minimum_nights = minimum_nights
        

In [16]:
import random

sample = listing_gz.loc[:, [ 'id', 'host_id', 'host_name', 'review_scores_rating', 'bedrooms', 'beds', 'bathrooms_text', 'neighbourhood_cleansed', 'price', 'accommodates', 'minimum_nights']]
sample.reset_index(drop=True, inplace=True)
sample.sort_values(by='review_scores_rating', ascending=False, inplace=True)
sample = sample.loc[:, :]
sample.reset_index(drop=True, inplace=True)
sample


,id,host_id,host_name,review_scores_rating,bedrooms,beds,bathrooms_text,neighbourhood_cleansed,price,accommodates,minimum_nights
0,922833637376956356,490272024,Daiki,5.0,1.0,1.0,1 bath,Koto Ku,"$9,600.00",2,1
1,727140790042920016,481595902,あずさ,5.0,1.0,2.0,1 bath,Sumida Ku,"$11,536.00",4,2
2,729355347138793951,481729050,Stay Support,5.0,1.0,1.0,1 shared bath,Shinjuku Ku,"$21,600.00",3,1
3,30049203,68181338,渥美,5.0,1.0,6.0,1 private bath,Arakawa Ku,"$25,000.00",7,1
4,30047622,68181338,渥美,5.0,1.0,2.0,0 shared baths,Arakawa Ku,"$9,000.00",2,1
...,...,...,...,...,...,...,...,...,...,...,...
11172,922873759575839340,294322772,鈴木,NaN,1.0,3.0,1 bath,Shinjuku Ku,"$12,000.00",4,2
11173,922998222764343634,518343589,Yingzi,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",3,2
11174,923008447744339896,518343589,Yingzi,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",4,2
11175,923011844205437846,518343589,Yingzi,NaN,3.0,6.0,1 bath,Shinjuku Ku,"$40,000.00",9,2


In [17]:
import csv 

user_list = []

for i in range(len(sample)):
    #price = sample['price'][i].replace('$', '').replace(',', '')
    user = User(idx = i, listing_id = sample['id'][i], host_id = sample['host_id'][i], host_name = sample['host_name'][i], review_scores_rating = sample['review_scores_rating'][i], bedrooms = sample['bedrooms'][i], beds = sample['beds'][i], bathrooms = sample['bathrooms_text'][i],region = sample['neighbourhood_cleansed'][i], price = sample['price'][i], accommodates = sample['accommodates'][i], minimum_nights = sample['minimum_nights'][i])
    user_list.append(user)

# CSV 파일에 데이터 쓰기
csv_filename = '/Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['idx', 'listing_id', 'host_id', 'host_name', 'review_scores_rating', 'bedrooms', 'beds', 'bathrooms','region', 'price', 'accommodates', 'minimum_nights']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # CSV 파일의 헤더 작성
    writer.writeheader()
    
    # 사용자 데이터를 CSV 파일에 작성
    for user in user_list:
        writer.writerow({
            'idx' : user.idx,
            'listing_id' : user.listing_id,
            'host_id' : user.host_id,
            'host_name' : user.host_name,
            'review_scores_rating' : user.review_scores_rating,
            'bedrooms' : user.bedrooms,
            'beds' : user.beds,
            'bathrooms' : user.bathrooms,
            'region': user.region,
            'price': user.price,
            'accommodates': user.accommodates,
            'minimum_nights': user.minimum_nights,
        })

print(f'사용자 데이터가 {csv_filename} 파일에 성공적으로 저장되었습니다.')

사용자 데이터가 /Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv 파일에 성공적으로 저장되었습니다.


In [18]:
user_db = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Pipeline/csv/user_totaldb.csv')
user_db

#* id는 하나만 있으므로 평점 높은 숙소 대상으로 정렬

,idx,listing_id,host_id,host_name,review_scores_rating,bedrooms,beds,bathrooms,region,price,accommodates,minimum_nights
0,0,922833637376956356,490272024,Daiki,5.0,1.0,1.0,1 bath,Koto Ku,"$9,600.00",2,1
1,1,727140790042920016,481595902,あずさ,5.0,1.0,2.0,1 bath,Sumida Ku,"$11,536.00",4,2
2,2,729355347138793951,481729050,Stay Support,5.0,1.0,1.0,1 shared bath,Shinjuku Ku,"$21,600.00",3,1
3,3,30049203,68181338,渥美,5.0,1.0,6.0,1 private bath,Arakawa Ku,"$25,000.00",7,1
4,4,30047622,68181338,渥美,5.0,1.0,2.0,0 shared baths,Arakawa Ku,"$9,000.00",2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11172,11172,922873759575839340,294322772,鈴木,NaN,1.0,3.0,1 bath,Shinjuku Ku,"$12,000.00",4,2
11173,11173,922998222764343634,518343589,Yingzi,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",3,2
11174,11174,923008447744339896,518343589,Yingzi,NaN,1.0,2.0,1 bath,Shinjuku Ku,"$16,000.00",4,2
11175,11175,923011844205437846,518343589,Yingzi,NaN,3.0,6.0,1 bath,Shinjuku Ku,"$40,000.00",9,2
